In [ ]:
import preprocessing.preprocessing as pp
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau



nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
df = pp.load_data("english_cleaned_lyrics.csv")
df = pp.preprocess_data(df)
df = pp.adjust_genre_distribution(df)
df = df[['lyrics', 'genre']]
X = df['lyrics'].to_list()
y = df['genre'].to_list()

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

In [ ]:
max_words = 25000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train_seq = pad_sequences(sequences, maxlen=max_len)

test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_seq = pad_sequences(test_sequences, maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words + 1, output_dim=20))
model.add(LSTM(15, dropout=0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train_seq, y_train, batch_size=32, epochs=10, validation_split=0.2)

In [ ]:
loss, accuracy = model.evaluate(X_test_seq, y_test)
print("Test Accuracy:", accuracy)